# Add calculator tool to math tutor agent

In [1]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
## llm configuration
llm_config = {
    "model": "gpt-4o",
    "api_key": os.getenv("OPENAI_API_KEY")
}

## Define a calculator tool

In [3]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

def calculator(a:int, b:int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return a / b
    else:
        raise ValueError("Invalid operator")

## Defining a Math tutor agent

In [6]:
# Let's first define the assistant agent that suggests tool calls
assistant = ConversableAgent(
    name="Assistant",
    system_message="You ate a helpful AI assistant. You can help with simple calculations. Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

## A user proxy to interact

In [7]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER"
)

## Registering the tools with the agents

In [9]:
# Register the tools signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

## Alternative way for registering tools

In [11]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,      # The assistant agent can suggest calls to the calculator
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name= "calculator",
    description="A simple Calculator",
)

/Users/ravishankarkushwaha/anaconda3/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3444: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/ravishankarkushwaha/anaconda3/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3342: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


## Initiate the conversation

In [13]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_GqUzr4esKEzgYwAsnBbgFZhJ): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_mGquBMHcbvJsYeDPsk8zadOl): calculator *****
Arguments: 
{"a": 13312, "b": 200, "operator": "/"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_GqUzr4esKEzgYwAsnBbgFZhJ
Input arguments: {'a': 232, 'b': 32, 'operator': '-'}

>>>>>>>> EXECUTED FUNCTION calculator...
Call ID: call_GqUzr4esKEzgYwAsnBbgFZhJ
Input arguments: {'a': 232, 'b': 32, 'operator': '-'}
Output:
200

>>>>>>>> EXECUTING FUNCTION cal